In [1]:
import pormake as pm
import numpy as np
import csv

db = pm.Database()
loc = pm.Locator()

## metal nodes
Fe_oct_a = pm.BuildingBlock(bb_file="pi-d_building_blocks/old_bb/Fe_oct.xyz")
Fe_oct_b = Fe_oct_a.make_chiral_building_block()
Fe_oct = Fe_oct_a

## organic nodes
B_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/B_O.xyz")
TP_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/TP_O.xyz")
TN_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/TN_O.xyz")
PcM_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/PcM_O.xyz")
PcMN_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/PcMN_O.xyz")

## edges
spacer = pm.BuildingBlock(bb_file="pi-d_building_blocks/spacer.xyz")
DHBQ = pm.BuildingBlock(bb_file="pi-d_building_blocks/DHBQ.xyz")
Cu_pln = pm.BuildingBlock(bb_file="pi-d_building_blocks/Cu_pln.xyz")
Fe_pln = pm.BuildingBlock(bb_file="pi-d_building_blocks/Fe_pln.xyz")
pseudo_edge = pm.BuildingBlock(bb_file="pi-d_building_blocks/pseudo_edge.xyz")

pm.log.disable_print()
pm.log.disable_file_print()


2023-07-27 17:35:13.514026: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 17:35:13.549203: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 17:35:13.550097: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 17:35:14.129615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
>>> There are atoms without bond: Fe_oct, [(4, 'Xx'), (5, 'Xx'), (6, 'Xx')].
>>> There are atoms without bond: B_O, [(16, 'Xx'), (17, 'Xx'), (15, 'Xx')].
>>> There are atoms without bond: TP_O, [(33, 'Xx'), (34, 'Xx'), (35, 'Xx')].
>>> There are atoms without bond: TN_O, [(51, 'Xx'),

## case 1: 2M 3,4L

In [2]:
keep_case1 = [ 'cdl',
 'cdm',
 'cdn',
 'cds',
 'cds-b',
 'cdz',
 'clh',
 'etc',
 'hof',
 'mot',
 'mou',
 'nbo',
 'nta',
 'ssd',
 'sse',
 'ssf',
 'sst',
 'tfc',
 'tfe',
 'tfn',
 'tfz',
 'ths']

In [4]:

test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=15, check_tetrahedral=True)

for topo_name in keep_case1:

    ## screen through all building blocks
    for node3 in [(B_O, 'B'), (TP_O, 'TP'), (TN_O, 'TN')]:
        for node4 in [(PcM_O, 'PcM'), (PcMN_O, 'PcMN')]:
            
            topo = db.get_topo(topo_name)
                   
            current_node = {}
            current_edge = {}   
            
            ## initialize edges
            for i, edge in enumerate(topo.unique_edge_types):
                current_edge[tuple(edge)] = Fe_pln
            edge_string = 'M2c_'

            ## initialize nodes
            namelist_nodes = []
            for i, node in enumerate(topo.unique_cn):                
                if node == 3:
                    current_node[i] = node3[0]
                    namelist_nodes.append(node3[1])
                elif node == 4:
                    current_node[i] = node4[0]
                    namelist_nodes.append(node4[1])
                else:
                    print('something wrong with', topo.name)
            nameset_nodes = set(namelist_nodes)
            node_string = ''
            for comp in nameset_nodes:
                node_string = node_string + comp + '_'
            node_string = node_string[:-1]
            
            bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
            calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
            calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
    
            try:
                mof = test_builder.build(topo, bbs, calc_permutation)        
                #mof.view()
                target_directory = 'raw_generated_MOFs/'
                cur_filename = topo.name + '_' + edge_string + node_string + '.cif'
                mof.atoms.write(target_directory + cur_filename, format='cif')
        
            except:
                print("build failed for " + topo.name + "!!!")
                continue


2023-07-27 17:35:59.002571: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/chong/miniconda3/envs/pormake/lib/python3.8/site-packages/pormake-0.0.2-py3.8.egg/pormake/locator.py:21: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)
/home/chong/miniconda3/envs/pormake/lib/python3.8/site-packages/pormake-0.0.2-py3.8.egg/pormake/builder.py:670: RuntimeWarning: invalid value encountered in scalar divide
  ratio = rmsd / slot_min_rmsd[key]
/home/chong/miniconda3/envs/pormake/lib/python3.8/site-packages/pormake-0.0.2-py3.8.egg/pormake/builder.py:680: RuntimeWarning: 

build failed for lwg!!!
build failed for lwg!!!
build failed for tta!!!
build failed for tta!!!


## Case 2: 3M, 2L

In [5]:
keep_case2 = [('bcu-h', 8, True),
 ('bpc', 2, True),
 ('eta', 6, True),
 ('etb', 3, False),
 ('etc', 1, False),
 ('etd', 12, True),
 ('lig', 4, False),
 ('noj', 6, True),
 ('pbz', 12, False),
 ('srs', 8, False),
 ('sxc-d', 12, False),
 ('ttg', 8, True),
 ('uni-d', 12, False),
 ('utg', 8, False)]

In [ ]:

test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=30, check_tetrahedral=False)

for case in keep_case2:

    for cur_edge in [(DHBQ, 'DHBQ')]:
    
        topo = db.get_topo(case[0])
        chiral_grouping = case[1]
        chiral_begin_flip = case[2]
    
        current_node = {}
        current_edge = {}    
    
        ## initialize nodes
        for i, node in enumerate(topo.unique_cn):
            current_node[i] = Fe_oct
        node_string = 'Feoct_'
            
        ## initialize edges
        for i, edge in enumerate(topo.unique_edge_types):
            current_edge[tuple(edge)] = cur_edge[0]
        edge_string = cur_edge[1]
    
        bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
        calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
        calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
    
        ## consider chirality of Fe_oct nodes
    
        chiral_counter = 0
        chiral_flip = chiral_begin_flip
    
        for node in topo.node_indices:
            
            chiral_counter += 1
    
            if chiral_flip:
                bbs[node] = Fe_oct_a
            else:
                bbs[node] = Fe_oct_b
                
            if chiral_counter % chiral_grouping == 0:
                chiral_flip = not chiral_flip
    
        try:
            mof = test_builder.build(topo, bbs, calc_permutation)        
            #mof.view()
            target_directory = 'raw_generated_MOFs/'
            cur_filename = topo.name + '_' + node_string + edge_string + '.cif'
            mof.atoms.write(target_directory + cur_filename, format='cif')
    
        except:
            print("build failed for " + topo.name + "!!!")
            continue


## Case 3: 3M, 4L

In [ ]:
keep_case3 = [('fjh', 4, False),
 ('fog', 3, True),
 ('iab', 4, False),
 ('jph', 8, False),
 ('lil', 4, True),
 ('pto', 4, True),
 ('stu', 8, False),
 ('sur', 2, True)]

In [ ]:

test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=45, check_tetrahedral=False)

for case in keep_case3:

    for cur_node in [(PcM_O, 'PcM'), (PcMN_O, 'PcMN')]:      
        for cur_ML_edge in [(pseudo_edge, ''), (spacer, 'spacer')]:

            topo = db.get_topo(case[0])
            chiral_grouping = case[1]
            chiral_begin_flip = case[2]
    
            current_node = {}
            current_edge = {}
        
            ## initialize nodes
            node_string = 'Feoct_'
            namelist_nodes = []                
            for i, node in enumerate(topo.unique_cn):
                if node == 3:
                    current_node[i] = Fe_oct_a
                if node == 4:
                    current_node[i] = cur_node[0]
                    namelist_nodes.append(cur_node[1])
            nameset_nodes = set(namelist_nodes)
            for comp in nameset_nodes:
                node_string = node_string + comp + '_'
            
            ## initialize edges
            for i, edge in enumerate(topo.unique_edge_types):
                current_edge[tuple(edge)] = spacer
        
            bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
            calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
            calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
            
            ## treat cn=4 cases. we do not have metal-based exp. cn=4 bbs, so enforce PcM linker for all cn=4 nodes.
            ## NOTE: For now, we leave out cases where cn=3 linkers are also mixed in.
        
            chiral_counter = 0
            chiral_flip = chiral_begin_flip
           
            for node in topo.node_indices:
        
                if topo.cn[node] == 4:
                    bbs[node] = PcM_O
        
                elif topo.cn[node] == 3:
                    
                    chiral_counter += 1
                    
                    if chiral_flip:
                        bbs[node] = Fe_oct_b
                    else:
                        bbs[node] = Fe_oct_a
                        
                    if chiral_counter % chiral_grouping == 0:
                        chiral_flip = not chiral_flip
                
        
            ## iterate through edges and assign edges accordingly
            namelist_edges = []            
            for edge in topo.edge_indices:
        
                node_a, node_b = topo.get_neighbor_indices(edge)
            
                ## metal-metal case
                if bbs[node_a].name == 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
                    bbs[edge] = DHBQ
                    namelist_edges.append('DHBQ')
        
                ## metal-linker case
                elif bbs[node_a].name == 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
        
                    bbs[edge] = cur_ML_edge[0]
                    ## take care of spacer 
                    calc_permutation[edge] = [1, 0]
                    namelist_edges.append(cur_ML_edge[1])
        
                ## linker-metal case
                elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
        
                    bbs[edge] = cur_ML_edge[0]
                    ## take care of spacer   
                    calc_permutation[edge] = [0, 1]
                    namelist_edges.append(cur_ML_edge[1])
        
                ## linker-linker case
                elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
                    bbs[edge] = Cu_pln
                    namelist_edges.append('Cupln')
                            
                else:
                    print('something\'s wrong!')

            nameset_edges = set(namelist_edges)
            if '' in nameset_edges:
                nameset_edges.remove('')
            edge_string = ''
            for comp in nameset_edges:
                edge_string = edge_string + comp + '_'
            if len(edge_string) > 0:
                edge_string = edge_string[:-1]
            if len(edge_string) == 0:
                node_string = node_string[:-1]
                
            try:
                mof = test_builder.build(topo, bbs, calc_permutation)        
                #mof.view()
                target_directory = 'raw_generated_MOFs/'
                cur_filename = topo.name + '_' + node_string + edge_string + '.cif'
                mof.atoms.write(target_directory + cur_filename, format='cif')
                
            except:
                print("build failed for " + topo.name + "!!!")
                continue


## Case 4: 3M, 3L

In [11]:
keep_case4 = [('etb', 6, False, 3, False),
 ('etg', 8, True, 4, False),
 ('eth', 8, False, 4, True),
 ('etj', 8, False, 4, False),
 ('lig', 1, True, 2, False),
 ('nod', 8, True, 2, False),
 ('srs', 1, False, 81, False),
 ('ths', 1, False, 2, False)]

In [12]:

test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=90, check_tetrahedral=False)

for case in keep_case4:

    for cur_node in [(B_O, 'B'), (TP_O, 'TP'), (TN_O, 'TN')]:      
        for cur_ML_edge in [(pseudo_edge, ''), (spacer, 'spacer')]:    
            
            topo = db.get_topo(case[0])
            type_grouping = case[1]
            type_begin_flip = case[2]    
            chiral_grouping = case[3]
            chiral_begin_flip = case[4]
            
            current_node = {}
            current_edge = {}    
            
            ## initialize nodes with TP_O first (no chirality dependence... is this good?)
            for i, node in enumerate(topo.unique_cn):
                current_node[i] = TP_O
        
            ## initialize edges
            for i, edge in enumerate(topo.unique_edge_types):
                current_edge[tuple(edge)] = spacer
        
            bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
            calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
            calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
            
            ## treat cn=4 cases. we do not have metal-based exp. cn=4 bbs, so enforce PcM linker for all cn=4 nodes.
            ## NOTE: For now, we leave out cases where cn=3 linkers are also mixed in.
        
            type_counter = 0
            chiral_counter = 0
        
            type_flip = type_begin_flip
            chiral_flip = chiral_begin_flip

            node_string = 'Feoct_' + cur_node[1] + '_'
            
            for node in topo.node_indices:
                
                type_counter += 1                
        
                if type_flip:
                    bbs[node] = cur_node[0]
                else:
                    bbs[node] = Fe_oct
                
                if type_counter % type_grouping == 0:
                    type_flip = not type_flip
                
                ## chirality
                if bbs[node].name == "Fe_oct":
                    
                    chiral_counter += 1                
                    
                    if chiral_flip:
                        bbs[node] = Fe_oct_b
                    else:
                        bbs[node] = Fe_oct_a
                    
                    if chiral_counter % chiral_grouping == 0:
                        chiral_flip = not chiral_flip
            
            ## iterate through edges and assign edges accordingly
            namelist_edges = []                        
            for edge in topo.edge_indices:
        
                node_a, node_b = topo.get_neighbor_indices(edge)
        
                ## metal-metal case
                if bbs[node_a].name == 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
                    bbs[edge] = DHBQ
                    namelist_edges.append('DHBQ')
        
                ## metal-linker case
                elif bbs[node_a].name == 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
        
                    bbs[edge] = cur_ML_edge[0]
                    ## take care of spacer 
                    calc_permutation[edge] = [1, 0]
                    namelist_edges.append(cur_ML_edge[1])
                    
                ## linker-metal case
                elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
        
                    bbs[edge] = cur_ML_edge[0]
                    ## take care of spacer             
                    calc_permutation[edge] = [0, 1]
                    namelist_edges.append(cur_ML_edge[1])
        
                ## linker-linker case
                elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
                    bbs[edge] = Cu_pln
                    namelist_edges.append('Cupln')
        
                else:
                    print('something\'s wrong!')
        
        
            nameset_edges = set(namelist_edges)
            if '' in nameset_edges:
                nameset_edges.remove('')
            edge_string = ''
            for comp in nameset_edges:
                edge_string = edge_string + comp + '_'
            if len(edge_string) > 0:
                edge_string = edge_string[:-1]
            if len(edge_string) == 0:
                node_string = node_string[:-1]

            
            try:
                mof = test_builder.build(topo, bbs, calc_permutation)        
                #mof.view()
                target_directory = 'raw_generated_MOFs/'
                cur_filename = topo.name + '_' + node_string + edge_string + '.cif'
                mof.atoms.write(target_directory + cur_filename, format='cif')
                
            except:
                print("build failed for " + topo.name + "!!!")
                continue


build failed for eth!!!
build failed for eth!!!
